# Impacto humano de Inundaciones (Flood)

## Importar librerías

In [140]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import re
from geopy.geocoders import Nominatim

## Cargar base de datos

02desastres_definitivo.csv

In [141]:
df = pd.read_csv('../../Data/02ParaLimpiar/02desastres_paralimpiar.csv', delimiter=';', encoding='utf-8')
df.head(10)

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected,Total Damages Adj
0,1900-9002-CPV,1900,9002,Climatological,Drought,Drought,Cabo Verde,CPV,Western Africa,Africa,...,NaN,1900,NaN,NaN,1900,NaN,NaN,11000.0,NaN,NaN
1,1900-9001-IND,1900,9001,Climatological,Drought,Drought,India,IND,Southern Asia,Asia,...,NaN,1900,NaN,NaN,1900,NaN,NaN,1250000.0,NaN,NaN
2,1902-0012-GTM,1902,12,Geophysical,Earthquake,Ground movement,Guatemala,GTM,Central America,Americas,...,-91,1902,4.0,18.0,1902,4.0,18.0,2000.0,NaN,843726.0
3,1902-0003-GTM,1902,3,Geophysical,Volcanic activity,Ash fall,Guatemala,GTM,Central America,Americas,...,NaN,1902,4.0,8.0,1902,4.0,8.0,1000.0,NaN,NaN
4,1902-0010-GTM,1902,10,Geophysical,Volcanic activity,Ash fall,Guatemala,GTM,Central America,Americas,...,NaN,1902,10.0,24.0,1902,10.0,24.0,6000.0,NaN,NaN
5,1903-0006-CAN,1903,6,Geophysical,Mass movement (dry),Rockfall,Canada,CAN,Northern America,Americas,...,NaN,1903,4.0,29.0,1903,4.0,29.0,76.0,23.0,NaN
6,1903-0012-COM,1903,12,Geophysical,Volcanic activity,Ash fall,Comoros (the),COM,Eastern Africa,Africa,...,NaN,1903,NaN,NaN,1903,NaN,NaN,17.0,NaN,NaN
7,1904-0003-BGD,1904,3,Meteorological,Storm,Tropical cyclone,Bangladesh,BGD,Southern Asia,Asia,...,NaN,1904,11.0,NaN,1904,11.0,NaN,NaN,NaN,NaN
8,1905-0005-CAN,1905,5,Geophysical,Mass movement (dry),Rockfall,Canada,CAN,Northern America,Americas,...,NaN,1905,8.0,13.0,1905,8.0,13.0,18.0,18.0,NaN
9,1905-0003-IND,1905,3,Geophysical,Earthquake,Ground movement,India,IND,Southern Asia,Asia,...,76.16,1905,4.0,4.0,1905,4.0,4.0,20000.0,NaN,812477.0


## EDA Inicial Básico

In [142]:
df.shape

(16636, 26)

In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16636 entries, 0 to 16635
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Dis No             16636 non-null  object 
 1   Year               16636 non-null  int64  
 2   Seq                16636 non-null  int64  
 3   Disaster Subgroup  16636 non-null  object 
 4   Disaster Type      16636 non-null  object 
 5   Disaster Subtype   13313 non-null  object 
 6   Country            16636 non-null  object 
 7   ISO                16636 non-null  object 
 8   Region             16636 non-null  object 
 9   Continent          16636 non-null  object 
 10  Location           14825 non-null  object 
 11  Origin             4085 non-null   object 
 12  Associated Dis     3593 non-null   object 
 13  Dis Mag Value      5064 non-null   float64
 14  Dis Mag Scale      15416 non-null  object 
 15  Latitude           2775 non-null   object 
 16  Longitude          277

## Filtro Disaster Type == Flood

In [144]:
filter = df['Disaster Type'] == 'Flood'
df_flood = df[filter]

In [145]:
df_flood.head(2)

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Country,ISO,Region,Continent,...,Longitude,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,Total Affected,Total Damages Adj
12,1906-0023-BEL,1906,23,Hydrological,Flood,NaN,Belgium,BEL,Western Europe,Europe,...,NaN,1906,5.0,14.0,1906,5.0,14.0,6.0,NaN,NaN
13,1906-0024-BEL,1906,24,Hydrological,Flood,NaN,Belgium,BEL,Western Europe,Europe,...,NaN,1906,4.0,NaN,1906,4.0,NaN,NaN,NaN,NaN


In [146]:
df_flood.shape

(5808, 26)

In [147]:
df_flood['Disaster Type'].dtype

dtype('O')

# Limpieza de coordenadas

## Función para limpiar coordenadas filtrado para df_flood

In [148]:
# Convertir 'Latitude' y 'Longitude' a string
df['Latitude'] = df['Latitude'].astype(str)
df['Longitude'] = df['Longitude'].astype(str)

# Limpiar las coordenadas latitude y longitude
df['Latitude'] = df['Latitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
df['Longitude'] = df['Longitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)

# Eliminar puntos finales
df['Latitude'] = df['Latitude'].str.rstrip('.')
df['Longitude'] = df['Longitude'].str.rstrip('.')

# Cambiar strings vacíos a NaN
df['Latitude'] = df['Latitude'].replace('', np.nan)
df['Longitude'] = df['Longitude'].replace('', np.nan)

# Convertir coordenadas válidas a float
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

# Identificar valores anómalos a partir de los grados
anomalous_lat = df['Latitude'].abs() > 90
anomalous_lon = df['Longitude'].abs() > 180

# Configurar valores anómalos a NaN
df.loc[anomalous_lat, 'Latitude'] = np.nan
df.loc[anomalous_lon, 'Longitude'] = np.nan

In [149]:
# df_flood = df[(df['Disaster Subtype'] == 'Flood') & (df['Total Deaths'].notna())]

# # Limpiar coordenadas latitude y longitude para df_flood
# df_flood['Latitude'] = df_flood['Latitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
# df_flood['Longitude'] = df_flood['Longitude'].apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)

# # Borrar puntos finales
# df_flood['Latitude'] = df_flood['Latitude'].str.rstrip('.')
# df_flood['Longitude'] = df_flood['Longitude'].str.rstrip('.')

# # Cambiar strings vacíos a NaN
# df_flood['Latitude'] = df_flood['Latitude'].replace('', np.nan)
# df_flood['Longitude'] = df_flood['Longitude'].replace('', np.nan)

# # Convertir coordenadas válidas a float
# df_flood['Latitude'] = df_flood['Latitude'].astype(float)
# df_flood['Longitude'] = df_flood['Longitude'].astype(float)

# # Remover filas con coordenadas NaN
# df_flood = df_flood.dropna(subset=['Latitude', 'Longitude'])


In [150]:
# import re
# import numpy as np

# # Filtrar el df_flood DataFrame para Disaster Subtype 'Flood' con non-null values en columnas específicas
# df_flood = df[(df['Disaster Subtype'] == 'Flood') & (df['Total Affected'].notna()) & (df['Total Deaths'].notna()) & (df['Dis Mag Scale'].notna()) & (df['Dis Mag Value'].notna())]

# # Limpiar y convertir la columna 'Latitude'
# df_flood['Latitude'] = df_flood['Latitude'].astype(str).apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
# df_flood['Latitude'] = df_flood['Latitude'].str.rstrip('.')
# df_flood['Latitude'] = df_flood['Latitude'].replace('', np.nan)
# df_flood['Latitude'] = df_flood['Latitude'].astype(float)

# # Limpiar y convertir la columna 'Longitude'
# df_flood['Longitude'] = df_flood['Longitude'].astype(str).apply(lambda x: re.sub('[^\d.-]', '', x.split('.', 1)[0]) if isinstance(x, str) else x)
# df_flood['Longitude'] = df_flood['Longitude'].str.rstrip('.')
# df_flood['Longitude'] = df_flood['Longitude'].replace('', np.nan)
# df_flood['Longitude'] = df_flood['Longitude'].astype(float)

# # Remover filas con NaN en coordenadas, 'Total Deaths', y 'Total Affected'
# df_flood = df_flood.dropna(subset=['Latitude', 'Longitude', 'Total Deaths', 'Total Affected'])


## Tests de verificación de limpieza de coordenadas

In [151]:
print(df_flood['Latitude'])
print(df_flood['Longitude'])

12       NaN
13       NaN
34       NaN
39       NaN
43       NaN
        ... 
16620    NaN
16628    NaN
16629    NaN
16631    NaN
16634    NaN
Name: Latitude, Length: 5808, dtype: object
12       NaN
13       NaN
34       NaN
39       NaN
43       NaN
        ... 
16620    NaN
16628    NaN
16629    NaN
16631    NaN
16634    NaN
Name: Longitude, Length: 5808, dtype: object


In [152]:
print(df_flood['Latitude'].dtype)
print(df_flood['Longitude'].dtype)

object
object


In [153]:
# Examinar filas específicas para verificar si el proceso de limpieza ha manejado los valores anómalos correctamente.
print(df_flood.loc[12, 'Latitude'])
print(df_flood.loc[12, 'Longitude'])

nan
nan


## Outliers de Latitude y Longitude

In [154]:
sns.boxplot(df_flood['Longitude'])

KeyError: 0

In [ ]:
sns.boxplot(df_flood['Latitude'])

## Valores faltantes

In [ ]:
df[['Longitude', 'Latitude']].isnull().sum()

# Afectados por Inundaciones

In [ ]:
df_flood['Total Deaths'].sum()

In [ ]:
df_flood['Total Affected'].sum()

## Nulos

In [ ]:
df_flood.isnull().sum()

In [ ]:
df_flood['Country'].value_counts()

In [ ]:
df_flood.groupby('Country')['Total Deaths'].sum().to_frame()

In [ ]:
total_deaths_by_continent = df_flood.groupby('Continent')['Total Deaths'].sum().to_frame()
total_deaths_by_continent

In [ ]:
total_deaths_by_region = df_flood.groupby('Continent')['Total Deaths'].sum().to_frame()
total_deaths_by_region

In [ ]:
total_deaths_by_country = df_flood.groupby('Country')['Total Deaths'].value_counts()
total_deaths_by_country

In [ ]:
df_flood[['Longitude', 'Latitude', 'Country']].reset_index().head(50)

In [ ]:
# plt.figure(figsize=(12,8), dpi=300)
# sns.swarmplot(data=df_flood, x='Longitude', y= 'Latitude', hue= 'Continent', palette='Paired', size='Total Deaths', sizes=(20,200))